<a href="https://colab.research.google.com/github/GregoryLavor/analise-lotacao/blob/main/Analise_Lotacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Análise de Dados utilizando PostgreSQL e Python (Pandas e PandaSQL)

##### Para esta análise utilizei o PostgreSQL como SGBD (Sistema de Gerenciamento de Banco de Dados) para o armazenamento dos dados em um banco de dados relacional e para gerar os arquivos das tabelas em CSV, para importação no ambiente Python.

#####  A leitura dos dados do PostgreSQL foi feita utilizando a biblioteca Pandas, que oferece recursos eficientes para manipulação de dados.

##### A biblioteca PandaSQL foi utilizada para realizar contultas SQL diretamente nos DataFrames do Pandas.

####Configurando o setup

In [ ]:
!pip install pandasql

####Importando as bibliotecas

In [ ]:
import pandas as pd
import pandasql as ps

####Lendo as bases de dados

In [ ]:
tb_cargo = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Cargo.csv')
tb_departamento = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Departamento.csv')
tb_funcionario = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Funcionario.csv')
tb_lotacao = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Lotacao.csv')

#### Problema de negócio

##### Os aspectos abordados incluem a distribuição de funcionários por faixa salarial, média salarial por cargo, idade dos funcionários, quantidade de funcionários com deficiência, entre outros.

##### Para tanto, foram realizadas diversas consultas SQL com a finalidade de extrair informações valiosas dos dados armazenados.

##### Cada consulta SQL foi cuidadosamente elaborada com a intensão de gerar insights relevantes e acionáveis para a tomada de decisão.

#### 1. Listar nome (funcionário e cargo) por departamento com salário e pretensão salarial.

In [227]:
query = """
select
cf.nome,
cc.nome as cargo,
cd.nome as departamento,
cf.pretensao_salarial,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
inner join tb_departamento as cd on cd.id = lt.id_departamento
order by departamento desc, salario desc
"""

ps.sqldf(query, locals())

,nome,cargo,departamento,pretensao_salarial,salario
0,Pedro,Analista de Dados,Tecnologia da Informação,5500.0,9000.0
1,Afonso,Analista de Dados,Tecnologia da Informação,5000.0,9000.0
2,Morgana,Analista de Infra,Tecnologia da Informação,5400.0,8000.0
3,Dorival,Analista de Infra,Tecnologia da Informação,3500.0,8000.0
4,Maria,Analista Contábil,Tecnologia da Informação,6500.0,5000.0
5,Lia,Analista Contábil,Tecnologia da Informação,5600.0,5000.0
6,Arthur,Estagiário,Tecnologia da Informação,1100.0,1100.0
7,João,Programador,Administrativo,6000.0,7500.0
8,Antonio,Programador,Administrativo,6500.0,7500.0
9,Julia,Gestor Financeiro,Administrativo,9000.0,7000.0


##### Insight: A análise mostra que dentro do departamento de Tecnologia da Informação, os cargos de Analista de Dados e Analista de Infraestrutura têm salários consideravelmente acima das pretensões salariais, enquanto o cargo de Analista Contábil é o único com salários abaixo das pretensões, sugerindo uma possível necessidade de revisão nas políticas salariais para esses cargos.

#### 2. Listar os funcionários (nome e salário) que não sejam PCD.

In [ ]:
print('Tabela Lotação')
display(tb_lotacao.head(2))
print('Tabela Funcionário')
display(tb_funcionario.head(2))

Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [224]:
query = """
select
cf.nome,
case when cf.pcd = 0 then 'nao' else 'sim' end as pcd,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
where cf.pcd = false
order by salario desc
"""

ps.sqldf(query, locals())

,nome,pcd,salario
0,Morgana,nao,8000.0
1,Dorival,nao,8000.0
2,João,nao,7500.0
3,Antonio,nao,7500.0
4,Julia,nao,7000.0
5,Lucio,nao,7000.0
6,Laura,nao,6500.0
7,Maria,nao,5000.0
8,Lia,nao,5000.0
9,Arthur,nao,1100.0


#### 3. Listar os cargos com suas respectivas médias salariais e média de idade.

In [ ]:
print('Tabela Cargo')
display(tb_cargo.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Cargo


,id,nome
0,1,Analista de Dados
1,2,Programador


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [220]:
query = """
select
cc.nome as cargo,
avg(lt.salario) as media_salario,
avg(date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento)))) as media_idade
from tb_lotacao as lt
inner join tb_cargo as cc on cc.id = lt.id_cargo
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
group by cargo
order by media_salario desc
"""
ps.sqldf(query,locals())

,cargo,media_salario,media_idade
0,Analista de Dados,9000.0,26.5
1,Analista de Infra,8000.0,27.0
2,Programador,7500.0,48.5
3,Gestor Financeiro,7000.0,19.5
4,Coordenador de Área,6500.0,38.5
5,Analista Contábil,5000.0,38.5
6,Estagiário,1100.0,23.0


#### 4. Listar os funcionários com salário e pretensão salarial, ordenando por quem recebe melhor em relação ao pretendido.

In [ ]:
print('Tabela Funcionário')
display(tb_funcionario.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [ ]:
query = """
select
cf.nome,
cc.nome as cargo,
cf.pretensao_salarial,
lt.salario,
lt.salario - cf.pretensao_salarial as diferenca,
case when salario > pretensao_salarial then 'Acima' else (case when salario = pretensao_salarial then 'Igual' else 'Abaixo' end) end as relacao
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
order by salario - pretensao_salarial desc
"""
ps.sqldf(query,locals())

,nome,cargo,pretensao_salarial,salario,diferenca,relacao
0,Dorival,Analista de Infra,3500.0,8000.0,4500.0,Acima
1,Afonso,Analista de Dados,5000.0,9000.0,4000.0,Acima
2,Pedro,Analista de Dados,5500.0,9000.0,3500.0,Acima
3,Morgana,Analista de Infra,5400.0,8000.0,2600.0,Acima
4,João,Programador,6000.0,7500.0,1500.0,Acima
5,Laura,Coordenador de Área,5400.0,6500.0,1100.0,Acima
6,Antonio,Programador,6500.0,7500.0,1000.0,Acima
7,Lucio,Gestor Financeiro,6000.0,7000.0,1000.0,Acima
8,Juvencio,Coordenador de Área,6250.0,6500.0,250.0,Acima
9,Arthur,Estagiário,1100.0,1100.0,0.0,Igual


##### Insight: A maioria dos funcionários recebe salários acima de suas pretensões, o que pode indicar uma política de remuneração competitiva na empresa. No entanto, algumas funcionários, como Maria, Julia e Lia, recebem salários abaixo de suas expectativas, o que pode afetar sua motivação e satisfação no trabalho.

#### 5. Listar quantos funcionários recebem acima, igual e abaixo do pretendido, com suas devidas porcentagens.

In [ ]:
query = """
select
sum(case when salario > pretensao_salarial then 1 else 0 end) as acima_pretendido,
round(cast(sum(case when salario > pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_acima,
sum(case when salario = pretensao_salarial then 1 else 0 end) as igual_pretendido,
round(cast(sum(case when salario = pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_igual,
sum(case when salario < pretensao_salarial then 1 else 0 end) as abaixo_pretendido,
round(cast(sum(case when salario < pretensao_salarial then 1 else 0 end) as real) * 100 / count(*) ,2) || '%' as perc_abaixo,
count(*) as total_funcionarios
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
"""
ps.sqldf(query,locals())

,acima_pretendido,perc_acima,igual_pretendido,perc_igual,abaixo_pretendido,perc_abaixo,total_funcionarios
0,9,69.23%,1,7.69%,3,23.08%,13


#### 6. Listar a pretensão salarial média e a média de salario por cargo, ordenando pelo cargo que recebe melhor em relação ao pretendido.

In [ ]:
query = """
select
cc.nome as cargo,
avg(cf.pretensao_salarial) as media_pretensao,
avg(lt.salario) as media_salario,
avg(lt.salario) - avg(cf.pretensao_salarial) as diferenca
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
group by cargo
order by diferenca desc
"""
ps.sqldf(query,locals())

,cargo,media_pretensao,media_salario,diferenca
0,Analista de Dados,5250.0,9000.0,3750.0
1,Analista de Infra,4450.0,8000.0,3550.0
2,Programador,6250.0,7500.0,1250.0
3,Coordenador de Área,5825.0,6500.0,675.0
4,Estagiário,1100.0,1100.0,0.0
5,Gestor Financeiro,7500.0,7000.0,-500.0
6,Analista Contábil,6050.0,5000.0,-1050.0


##### Insight: Os cargos de Analista Contábil e de Gestor Financeiro são os menos valorizados com relação a pretensão salarial. Isso pode indicar uma possível insatisfação e desmotivação dos funcionários.

#### 7. Listar a idade dos funcionários e classificá-los por faixa etária.

In [123]:
print('Tabela Funcionários')
display(tb_funcionario.head(2))

Tabela Funcionários


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [210]:
query = """
select
cf.nome,
cf.nascimento,
date('now') as data_atual,
date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) as idade
from tb_funcionario as cf
order by idade asc
"""
ps.sqldf(query,locals())

,nome,nascimento,data_atual,idade
0,Afonso,2009-07-02,2024-04-22,14
1,Lucio,2004-06-04,2024-04-22,19
2,Julia,2003-08-23,2024-04-22,20
3,Dorival,2003-09-01,2024-04-22,20
4,Laura,2001-09-05,2024-04-22,22
5,Maria,2000-05-03,2024-04-22,23
6,Arthur,2000-12-05,2024-04-22,23
7,Morgana,1990-04-07,2024-04-22,34
8,Pedro,1984-05-29,2024-04-22,39
9,João,1980-02-01,2024-04-22,44


In [128]:
query = """
select
case when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) < 25 then '0 a 25 anos'
when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) between 26 and 35 then '26 a 35 anos'
when date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) between 36 and 45 then '36 a 45 anos'
else 'mais de 45 anos' end as faixa_etaria,
count(*) as qtde_funcionarios
from tb_funcionario as cf
group by faixa_etaria
order by faixa_etaria asc
"""
ps.sqldf(query,locals())

,faixa_etaria,qtde_funcionarios
0,0 a 25 anos,7
1,26 a 35 anos,1
2,36 a 45 anos,2
3,mais de 45 anos,3


##### Insight: A empresa possui uma boa divisão no que diz respeito a faixa etária, possuindo uma porcentagem consideravel abaixo dos 25 anos e também acima dos 35. Isso indica uma distribuição etária diversificada na empresa, o que pode trazer diferentes perspectivas e experiências para a equipe.

#### 8. Listar os funcionários com deficiência, seus salários e média de idade.

In [129]:
print('Tabela Funcionários')
display(tb_funcionario.head(2))

Tabela Funcionários


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [222]:
query = """
select
cf.nome,
case when cf.pcd = true then 'sim' else 'nao' end as pcd,
lt.salario,
date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) as idade
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
where pcd = true
"""
ps.sqldf(query,locals())

,nome,pcd,salario,idade
0,Pedro,sim,9000.0,39
1,Afonso,sim,9000.0,14
2,Juvencio,sim,6500.0,55


##### Insight: Os maiores salários da empresa são de funcionárois com deficiência.

#### 9. Listar a quantidade de funcionários com deficiência e a devida porcentagem em relação ao total de funcionários.

In [141]:
query = """
select
sum(cf.pcd) as qtde_pcd,
round(cast(sum(cf.pcd) as real) * 100 / count(*) ,2) || '%' as perc_pcd,
count(*) as qtde_funcionarios
from tb_funcionario as cf
"""
ps.sqldf(query,locals())

,qtde_pcd,perc_pcd,qtde_funcionarios
0,3,23.08%,13


##### Insight: A empresa possui 3 funcionários com deficiência, representando aproximadamente 23.08% do total de funcionários. Isso pode indicar um compromisso com a inclusão e diversidade no ambiente de trabalho.

#### 10. Listar o tempo de empresa dos funcionários e ordenar pelo maior tempo de empresa.

In [190]:
query = """
select
cf.nome,
lt.cadastro,
date('now') as data_atual,
cast(date('now') - date(lt.cadastro) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(lt.cadastro))) as text) || ' ano(s) e ' ||
cast(case when strftime('%m',date('now')) <= strftime('%m',date(lt.cadastro)) then strftime('%m',date('now')) - strftime('%m',date(lt.cadastro)) + 12
else strftime('%m',date('now')) - strftime('%m',date(lt.cadastro)) end as texte) || ' meses' as tempo_de_servico
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
order by tempo_de_servico desc,cadastro asc
"""
ps.sqldf(query,locals())

,nome,cadastro,data_atual,tempo_de_servico
0,Arthur,2000-12-05,2024-04-22,23 ano(s) e 4 meses
1,Pedro,2022-09-01,2024-04-22,1 ano(s) e 7 meses
2,João,2022-09-02,2024-04-22,1 ano(s) e 7 meses
3,Maria,2022-09-05,2024-04-22,1 ano(s) e 7 meses
4,Laura,2022-09-06,2024-04-22,1 ano(s) e 7 meses
5,Morgana,2022-09-07,2024-04-22,1 ano(s) e 7 meses
6,Julia,2022-09-08,2024-04-22,1 ano(s) e 7 meses
7,Afonso,2022-09-09,2024-04-22,1 ano(s) e 7 meses
8,Antonio,2022-09-12,2024-04-22,1 ano(s) e 7 meses
9,Lia,2022-09-13,2024-04-22,1 ano(s) e 7 meses


#### 11. Listar a quantidade de funcionários por faixa salarial com suas devidas porcentagens.

In [209]:
query = """
select
case when lt.salario <= 2500 then 'até 2500,00'
when lt.salario between 2500 and 5000 then 'de 2500,01 a 5000,00'
when lt.salario between 5000 and 7500 then 'de 5000,01 a 7500,00'
else 'mais de 7500,00' end as faixa_salarial,
count(*) as qtde_funcionarios,
round(cast(count(*) as real) * 100 / (select count(*) from tb_lotacao), 2) || '%' as perc_funcionarios
from tb_lotacao as lt
group by faixa_salarial
"""
ps.sqldf(query,locals())

,faixa_salarial,qtde_funcionarios,perc_funcionarios
0,"até 2500,00",1,7.69%
1,"de 2500,01 a 5000,00",2,15.38%
2,"de 5000,01 a 7500,00",6,46.15%
3,"mais de 7500,00",4,30.77%


##### Insight: A grande maioria dos funcionários recebe salários acima de 5000,00. Isso pode indicar uma política de remuneração competitiva, que atrai e retém talentos qualificados na empresa.

#### 12. Listar a idade e o salário dos funcionários.

In [214]:
query = """
select
cf.nome,
date('now') - date(cf.nascimento) - (strftime('%m-%d',date('now')) < strftime('%m-%d',date(cf.nascimento))) as idade,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
order by salario desc,idade desc
"""
ps.sqldf(query,locals())

,nome,idade,salario
0,Pedro,39,9000.0
1,Afonso,14,9000.0
2,Morgana,34,8000.0
3,Dorival,20,8000.0
4,Antonio,53,7500.0
5,João,44,7500.0
6,Julia,20,7000.0
7,Lucio,19,7000.0
8,Juvencio,55,6500.0
9,Laura,22,6500.0


##### Insight: A análise mostra uma variedade de idades entre os funcionários, com os mais jovens ganhando salários mais altos do que alguns funcionários mais velhos. Isso pode indicar uma valorização das habilidades e experiências dos funcionários, independentemente da idade.